In [1]:
# !pip install transformers accelerate
from transformers import AutoTokenizer, AutoModel, EarlyStoppingCallback, AutoModelForSequenceClassification, AutoConfig,Trainer, TrainingArguments,DataCollatorWithPadding
import torch
import numpy as np
import pandas as pd
# !pip install datasets
# from datasets import load_metric
# !pip install evaluate
from evaluate import evaluator
from sklearn.model_selection import train_test_split
import csv

# notebook_login()

CUDA_LAUNCH_BLOCKING=1


In [2]:
# from huggingface_hub import notebook_login
# notebook_login()

In [3]:
df = pd.read_csv(r'/root/autodl-tmp/CommitFit/dataset/multi-lang-exp/dataset.csv', index_col=0, encoding='utf_8_sig')
df.fillna('', inplace=True)
label2id={'A':0, 'C':1, 'P':2}
df = df.rename(columns={'maintenance_type':'labels','commit_message':'message','commit_diff':'diffs'})
df = df.replace({"label": label2id})
# df = df['diffs']
# # print(df)
df
# test_sample = df.sample(n=3, random_state=1)

,language,commit,commit_url,labels,user,repo,feature,diffs,msgs
0,C++,be937a3290223f926fe50684f1344569a573ed4b,https://github.com/tensorflow/tensorflow/commi...,C,tensorflow,tensorflow,"[1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 6, ...",diff --git a/tensorflow/compiler/mlir/lite/fla...,Fixing per axis quantization bug in flatbuffer...
1,C++,c800d2e36954edddcb83aa1df7f623f2780c7780,https://github.com/tensorflow/tensorflow/commi...,A,tensorflow,tensorflow,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",diff --git a/tensorflow/python/tools/saved_mod...,Add support for python expression as SavedMode...
2,C++,469e56eeff17eb857c95e935ca7b49723c43470e,https://github.com/tensorflow/tensorflow/commi...,P,tensorflow,tensorflow,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",diff --git a/.github/ISSUE_TEMPLATE/00-bug-iss...,Fix GitHub issue templates. Actual fix for #36...
3,C++,24f5d302c0906f3377f4d6bcc2735d943ce13e3d,https://github.com/tensorflow/tensorflow/commi...,A,tensorflow,tensorflow,"[3, 37, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",diff --git a/tensorflow/compiler/xla/pjrt/dist...,"Add new PjRT distributed APIs (KeyValueDelete,..."
4,C++,1effdb7b959503596b07f7f3e74618ab63f9e5e5,https://github.com/tensorflow/tensorflow/commi...,P,tensorflow,tensorflow,"[1, 0, 12, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",diff --git a/tensorflow/core/common_runtime/ea...,Remove IsLocalDevice() since it's no longer us...
...,...,...,...,...,...,...,...,...,...
1576,Java,e4bbfd537a63a52a6094491504fc7b0bb54bcb0e,,C,kiegroup,drools,"[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",diff --git a/drools-persistence-jpa/src/main/j...,fixing reflection constructor to use int inste...
1577,Java,75b81dcbd26a11d5b370ea5bdf102c2d62ebd0c3,,C,kiegroup,drools,"[2, 6, 2, 3, 0, 0, 0, 1, 7, 0, 0, 0, 0, 0, 0, ...",diff --git a/drools-compiler/src/main/java/org...,Fix OutOfBoundException on- MemoryFileSystem.e...
1578,Java,6bc9677bbe919cd5bcadd8af2f5b0c838757a4ce,,C,kiegroup,drools,"[1, 1, 5, 13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",diff --git a/drools-core/src/main/java/org/dro...,[DROOLS-293] fix ObjectTypeNode id creation an...
1579,Java,48701edad8513b27acec7216581e64637157c86a,,A,kiegroup,drools,"[2, 24, 2, 4, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",diff --git a/drools-core/src/main/java/org/dro...,DROOLS-515 Kie-Camel is broken after Camel Upd...


In [4]:
train, temp_df = train_test_split(df, test_size=0.3, random_state=42)
val, test = train_test_split(temp_df, test_size=0.5, random_state=42)

In [5]:
# train.to_csv('/root/autodl-tmp/COLARE-main/dataset/1793/train.csv',index=False)
# test.to_csv('/root/autodl-tmp/COLARE-main/dataset/1793/test.csv',index=False)
# val.to_csv('/root/autodl-tmp/COLARE-main/dataset/1793/eval.csv',index=False)

In [6]:
def generate_samples(df, n=5):
    new_samples = []

    for label, group in df.groupby("labels"):
        for i in range(n):
            sample = {
                "labels": label,
                "msgs": f"This is a {label} commit",   # 模板生成
                "diffs": "This is a {label} commit"    # 可以留空，或随便生成占位
            }
            new_samples.append(sample)

    return pd.DataFrame(new_samples)

# 生成 n 个样本（每类 n 个）
synthetic_df = generate_samples(df, n=8)

# 合并到训练集
train = pd.concat([train, synthetic_df], ignore_index=True)

print(train)

        language                                    commit  \
0           Rust  e0b52079daeec08e5d349d686e217c8ba11c1f64   
1     JavaScript  ddff0f642acbf650b53d1ddffe544900569ab983   
2     JavaScript  57f2d7de58427a413c406ef0a5a4e120e3351620   
3           Rust  b00e9fc227dc2ab1070326f29fbe9626702dfa67   
4     TypeScript  c168a92f187943a401f6c4095b5abde44478d0c9   
...          ...                                       ...   
1125         NaN                                       NaN   
1126         NaN                                       NaN   
1127         NaN                                       NaN   
1128         NaN                                       NaN   
1129         NaN                                       NaN   

                                             commit_url labels          user  \
0     https://github.com/bevyengine/bevy/commit/e0b5...      A    bevyengine   
1     https://github.com/mrdoob/three.js/commit/ddff...      P        mrdoob   
2     https://g

In [7]:
len(train)

1130

In [8]:
train['labels'].value_counts()

labels
P    699
C    250
A    181
Name: count, dtype: int64

In [9]:
test['labels'].value_counts()

labels
P    155
C     51
A     32
Name: count, dtype: int64

In [10]:
# !pip install sentence_transformers
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('../../sentence-transformers/paraphrase-mpnet-base-v2')

In [11]:
train['embeddings'] = train['diffs'].apply(model.encode)
test['embeddings'] = test['diffs'].apply(model.encode)

In [12]:
test

,language,commit,commit_url,labels,user,repo,feature,diffs,msgs,embeddings
1551,Java,439b7750d4750722a03d60465da2e4b62bbcf626,,C,spring-projects,spring-framework,"[1, 5, 0, 1, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",diff --git a/org.springframework.orm/src/main/...,"fixed ""hibernateManagedSession"" mode to actual...","[-0.08523223, 0.4753142, 0.00838813, 0.0342326..."
31,C++,67817d4e45fe5640c971403dbde8be347b71f12a,https://github.com/microsoft/terminal/commit/6...,P,microsoft,terminal,"[1, 14, 69, 8, 13, 21, 3, 0, 0, 0, 0, 0, 0, 0,...",diff --git a/src/cascadia/TerminalSettingsMode...,Consolidate IInheritable macros (#11924)\n\nCo...,"[-0.12605341, 0.077841125, -0.028077705, 0.056..."
65,C++,f196dd5ff7871118eb42738f1ef5f771b0b9529c,https://github.com/opencv/opencv/commit/f196dd...,A,opencv,opencv,"[3, 231, 16, 28, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0...",diff --git a/modules/imgproc/include/opencv2/i...,preliminary version of a multi-level findconto...,"[-0.08439063, -0.06631157, 0.005969405, 0.0884..."
1027,Rust,75c8e09d6d412e1012a921d9f3932d6f253781e6,https://github.com/swc-project/swc/commit/75c8...,A,swc-project,swc,"[4, 6, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",diff --git a/crates/swc_css_codegen/src/writer...,feat(api/rust): Implement more traits for publ...,"[-0.08332649, 0.3296717, -0.057861865, 0.00249..."
111,C++,707ee709663ce7e8faac5345d4007f205379187a,https://github.com/tesseract-ocr/tesseract/com...,P,tesseract-ocr,tesseract,"[1, 4, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",diff --git a/src/training/pango_font_info.cpp ...,Use deprecated pango_fc_font_get_glyph for old...,"[-0.04197979, 0.18034188, 0.044548597, 0.05202..."
...,...,...,...,...,...,...,...,...,...,...
1102,Rust,8b25f44f0570c7b430d632108126a91f1939de65,https://github.com/helix-editor/helix/commit/8...,P,helix-editor,helix,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",diff --git a/Cargo.lock b/Cargo.lock\nindex f7...,build(deps): bump toml from 0.6.0 to 0.7.0 (#5...,"[-0.10873168, 0.2973023, -0.018619511, 0.05226..."
1118,TypeScript,850555d1faa9be7d8306adffd95c7dee5e58717f,https://github.com/vuejs/vue/commit/850555d1fa...,C,vuejs,vue,"[3, 17, 29, 17, 19, 9, 15, 1, 0, 0, 0, 0, 0, 1...",diff --git a/src/core/util/next-tick.js b/src/...,fix: always use microtasks for nextTick (#8450...,"[-0.0642169, -0.16394633, -0.057304256, 0.0876..."
1175,TypeScript,7dfdb815e40d79e857f5d2e6a7c1519428bbf058,https://github.com/mui/material-ui/commit/7dfd...,P,mui,material-ui,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",diff --git a/.git-blame-ignore-revs b/.git-bla...,[docs] Add warning about git-blame-ignore-revs...,"[-0.18914662, -0.17303096, 0.07548264, 0.05755..."
354,Go,e2535233bb539852c385271d8959a9ec16267cc1,https://github.com/caddyserver/caddy/commit/e2...,P,caddyserver,caddy,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",diff --git a/.github/workflows/release.yml b/....,fix typo (#4702)\n\nSigned-off-by: cuishuang <...,"[-0.016506122, 0.092044294, -0.03672568, 0.015..."


In [13]:
X_train = train['embeddings'].to_list()
X_test = test['embeddings'].to_list()

y_train = train['labels'].to_list()
y_test = test['labels'].to_list()

In [14]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(X_train,y_train)

LogisticRegression()

In [15]:
from sklearn import metrics
predicted = LR.predict(X_test)

print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted,average='macro'))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted,average='macro'))
print("Logistic Regression Recall:",metrics.f1_score(y_test, predicted,average='macro'))
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))

Logistic Regression Precision: 0.5603681188787571
Logistic Regression Recall: 0.4786514336917563
Logistic Regression Recall: 0.49735166562560557
Logistic Regression Accuracy: 0.6764705882352942


In [16]:
print("Logistic Regression Report:",metrics.classification_report(y_test, predicted,digits=4))

Logistic Regression Report:               precision    recall  f1-score   support

           A     0.4667    0.2188    0.2979        32
           C     0.4857    0.3333    0.3953        51
           P     0.7287    0.8839    0.7988       155

    accuracy                         0.6765       238
   macro avg     0.5604    0.4787    0.4974       238
weighted avg     0.6414    0.6765    0.6450       238



In [17]:
cm = metrics.confusion_matrix(y_test, predicted)
print("Confusion Matrix:\n", cm)

Confusion Matrix:
 [[  7   5  20]
 [  3  17  31]
 [  5  13 137]]
